In [9]:
import re, string, unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary


nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inlin

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import data and prepare data

In [10]:
# Load dataset
df = pd.read_excel('C:/Users/Kornelius/Desktop/Data 2/ecbpr/Warin_Sanger_ECB.xlsx') 
df.head()

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


,Date,Press,qa
0,1998,"Ladies and gentlemen, in line with our stated ...",We have no indications based on the most recen...
1,1998,"Ladies and gentlemen, as in previous months, t...",The answer to the first question is that it is...
2,1998,"Ladies and gentlemen, the Vice-President and I...",We simply felt that the announcement of a rang...
3,1998,"Ladies and gentlemen, the Vice-President and I...","No, that is not a signal that it is supposed t..."
4,1999,"Ladies and gentlemen, the Vice-President and I...",The main problems have not been technical ones...


In [11]:
#df = df.values.tolist()
# Drop a row by condition
# Print out the first rows of papers
#df = df[df.press]
df = df['qa']

In [12]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(df):
    stop_free = " ".join([i for i in df.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

data = [clean(df).split()for df in df]  

In [13]:
# remove characters and numbers
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [14]:
data = data_words
# Create Dictionary
id2word = corpora.Dictionary(data)
print((id2word))

Dictionary(8371 unique tokens: ['absence', 'accept', 'achieve', 'activity', 'add']...)


In [15]:
texts = data
# less than 10 documents,
id2word = corpora.Dictionary(texts)
id2word.filter_extremes(no_below = 50)

In [16]:
# Create Corpus
texts = data
# Term Document Frequency and creating corpus
corpus = [id2word.doc2bow(text) for text in texts]

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
This is used as the input by the LDA model.

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [17]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=1, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
lda_model5 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model20 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model50 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model100 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=100, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [19]:
# Print the Keyword in the 1 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"programme" + 0.011*"condition" + 0.011*"objective" + '
  '0.011*"actually" + 0.010*"recovery" + 0.009*"lending" + 0.009*"sense" + '
  '0.009*"bond" + 0.009*"basically" + 0.008*"several"')]


In [20]:
# Print the Keyword in the 5 topics
pprint(lda_model5.print_topics())
doc_lda = lda_model5[corpus]

# Print the Keyword in the 10 topics
pprint(lda_model10.print_topics())
doc_lda = lda_model10[corpus]

# Print the Keyword in the 20 topics
pprint(lda_model20.print_topics())
doc_lda = lda_model20[corpus]

# Print the Keyword in the 50 topics
pprint(lda_model50.print_topics())
doc_lda = lda_model50[corpus]

# Print the Keyword in the 100 topics
pprint(lda_model100.print_topics())
doc_lda = lda_model100[corpus]

[(0,
  '0.015*"week" + 0.011*"forecast" + 0.011*"mr" + 0.011*"rather" + '
  '0.010*"minister" + 0.010*"conclusion" + 0.009*"day" + 0.009*"earlier" + '
  '0.009*"move" + 0.009*"treaty"'),
 (1,
  '0.022*"money" + 0.020*"operation" + 0.015*"bond" + 0.014*"don" + '
  '0.013*"collateral" + 0.012*"programme" + 0.012*"crisis" + 0.011*"capital" + '
  '0.010*"billion" + 0.010*"actually"'),
 (2,
  '0.039*"programme" + 0.032*"bond" + 0.027*"committee" + 0.023*"purchase" + '
  '0.019*"reflect" + 0.019*"lending" + 0.018*"condition" + 0.018*"negative" + '
  '0.017*"loan" + 0.016*"sense"'),
 (3,
  '0.023*"programme" + 0.020*"objective" + 0.017*"recovery" + '
  '0.017*"condition" + 0.016*"actually" + 0.016*"basically" + 0.015*"several" '
  '+ 0.015*"sense" + 0.015*"uncertainty" + 0.015*"especially"'),
 (4,
  '0.015*"deliver" + 0.011*"currency" + 0.011*"observed" + 0.010*"staff" + '
  '0.009*"credible" + 0.009*"extremely" + 0.008*"creation" + 0.008*"quarter" + '
  '0.008*"behalf" + 0.008*"inflationary"

  '0.003*"treaty" + 0.003*"pact"'),
 (5,
  '0.003*"note" + 0.003*"environment" + 0.003*"deliver" + 0.003*"treaty" + '
  '0.003*"creation" + 0.003*"commitment" + 0.003*"credibility" + '
  '0.003*"potential" + 0.003*"december" + 0.003*"staff"'),
 (1,
  '0.003*"data" + 0.003*"treaty" + 0.003*"single" + 0.003*"concept" + '
  '0.003*"staff" + 0.003*"move" + 0.003*"inflationary" + 0.003*"evolution" + '
  '0.003*"observation" + 0.003*"discus"'),
 (47,
  '0.003*"conclusion" + 0.003*"dollar" + 0.003*"reserve" + '
  '0.003*"unemployment" + 0.003*"inflationary" + 0.003*"rather" + '
  '0.003*"around" + 0.003*"mr" + 0.003*"hope" + 0.003*"enough"'),
 (22,
  '0.350*"balance" + 0.281*"sheet" + 0.119*"we" + 0.096*"commitment" + '
  '0.022*"very" + 0.019*"eurosystem" + 0.015*"read" + 0.013*"several" + '
  '0.013*"asset" + 0.011*"amount"'),
 (37,
  '0.044*"forecast" + 0.026*"rather" + 0.026*"move" + 0.022*"precise" + '
  '0.021*"earlier" + 0.020*"every" + 0.019*"movement" + 0.019*"reference" + '
  '0.019

# Evaluation
Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be? You may summarise it either are ‘cars’ or ‘automobiles’.

In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity5: ', lda_model5.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity10: ', lda_model10.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity20: ', lda_model20.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity50: ', lda_model50.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity100: ', lda_model100.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100: ', coherence_lda)


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence ScoreUMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100UMass: ', coherence_lda)


Perplexity:  -6.015745148546349

Perplexity5:  -5.835293062096349

Perplexity10:  -5.881629974572767

Perplexity20:  -5.906556348152405

Perplexity50:  -5.9506385748876625

Perplexity100:  -6.026948155168523

Coherence Score:  0.461265452670076

Coherence Score5:  0.47002688034956125

Coherence Score10:  0.3966798354262531

Coherence Score20:  0.33648010878479384

Coherence Score50:  0.306563114210897

Coherence Score100:  0.28415343519753544

Coherence ScoreUMass:  -0.670171037231952

Coherence Score5UMass:  -0.6893683189891364

Coherence Score10UMass:  -0.7896978748807758

Coherence Score20UMass:  -0.8489323814462632

Coherence Score50UMass:  -0.8783749229539787

Coherence Score100UMass:  -0.8867189248937105
